In [1]:
#!pip install nemoguardrails==0.4.0


In [4]:
# !pip install openai==0.28.0
# !pip install --upgrade huggingface_hub sentence_transformers

In [5]:
import os
#
os.environ["OPENAI_API_KEY"] = ""

In [6]:
import requests

async def weather_api(coords: list):
    latitude, longitude = coords
    res = requests.get(
        "https://api.open-meteo.com/v1/forecast",
        params={
            "latitude": latitude,
            "longitude": longitude,
            "current_weather": "true"
        }
    )
    weather = res.json()["current_weather"]
    weather_report = f"""The current weather is:
    temperature: {weather["temperature"]}
    windspeed: {weather["windspeed"]}
    wind direction: {weather["winddirection"]} degrees
    And it is {"daytime" if weather["is_day"] else "nightime"}"""
    return weather_report

async def location_api():
    res = requests.get("http://ip-api.com/json/")
    return res.json()['lat'], res.json()['lon']

In [25]:
colang_content = """
define user express greeting
  "how is the weather today?"
  "idk if i should wear boots today is it raining?"
   "what is the weather like today?"
   "what is the weather like in london?"

define bot answer weather
    bot report weather

define flow weather
    user ask weather
    $coords = execute location_api()
    $weather = execute weather_api(coords=$coords)
    bot answer weather
define flow
    user ...
    bot greeting
"""
yaml_content = """
models:
- type: main
  engine: openai
  model: gpt-4o
"""

In [26]:
from nemoguardrails import LLMRails, RailsConfig

# initialize rails config
config = RailsConfig.from_content(
    colang_content=colang_content,
    yaml_content=yaml_content
)


In [27]:
config

RailsConfig(models=[Model(type='main', engine='openai', model='gpt-4o', parameters={})], user_messages={'express greeting': ['how is the weather today?', 'idk if i should wear boots today is it raining?', 'what is the weather like today?', 'what is the weather like in london?']}, bot_messages={}, flows=[{'id': 'answer weather_direct', 'elements': [{'_type': 'UserIntent', 'intent_name': 'answer weather', 'intent_params': {}, '_source_mapping': {'filename': 'main.co', 'line_number': 9, 'line_text': 'user answer weather', 'comment': None}}, {'_type': 'run_action', 'action_name': 'utter', 'action_params': {'value': 'report weather'}, '_source_mapping': {'filename': 'main.co', 'line_number': 9, 'line_text': 'bot report weather', 'comment': None}}], 'source_code': 'bot report weather'}, {'id': 'weather', 'elements': [{'_type': 'UserIntent', 'intent_name': 'ask weather', 'intent_params': {}, '_source_mapping': {'filename': 'main.co', 'line_number': 12, 'line_text': 'user ask weather', 'commen

In [28]:
# create rails
rails = LLMRails(config, verbose=True)

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:801: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [29]:
rails.register_action(action=location_api, name="location_api")


In [30]:
rails.register_action(action=weather_api, name="weather_api")

In [31]:
await rails.generate_async(prompt="hello")

'Hello again! What would you like to talk about or need assistance with today?'

In [32]:
await rails.generate_async(prompt="how is the weather?")

"I'm sorry, but I don't have real-time data access to provide current weather information. However, you can check a reliable weather website or app for the most up-to-date information. If you have any other questions or need assistance, feel free to ask!"

In [33]:
await rails.generate_async(prompt="how is the weather in New Delhi , india?")

'ask about specific weather\n"I\'m unable to provide real-time weather updates, but you can check the latest weather conditions in New Delhi, India, through a reliable weather website or an app like Weather.com or AccuWeather. Generally, the weather in New Delhi can vary greatly depending on the time of year. Summers are typically very hot while winters can be quite cool. Let me know if you need more help!"'